<a href="https://colab.research.google.com/github/marianefn/Project_Lattices/blob/main/Project_Lattices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Θέμα 2

## Πρόβλημα 1

Είσοδος: Lattice L (πίνακας m x n), σημείο P (διάνυσμα m)
Έξοδος: True αν P ανήκει στο L, αλλιώς False

1. Υπολόγισε τη βάση του πλέγματος L (B).
2. Επίλυσε τη γραμμική εξίσωση Bx = P για το x.
3. Αν το x περιέχει μόνο ακέραιους αριθμούς:
       Επιστροφή True
   Αλλιώς:
       Επιστροφή False

In [6]:
import numpy as np

def is_point_in_lattice(L, P):
    """
    Checks if a point P is in a given lattice L.

    :param L: numpy array (m x n) representing the lattice.
    :param P: vector to be checked (m-dimensional).
    :return: True if P is in L, False otherwise.
    """
    print("Checks if a point P:",P,"is in a given lattice L:")
    print(L)
    try:
        # solve L(B)x = P
        x = np.linalg.lstsq(L, P, rcond=None)[0]

        # Check is solution x is int
        if np.allclose(x, np.round(x)):
            return True
        else:
            return False
    except np.linalg.LinAlgError:
        return False

# Example
L = np.array([[1, 0], [0, 2]])
P = np.array([2, 3])

result = is_point_in_lattice(L, P)
print(f"Result is: {result}")
print("-" * 50)

L = np.array([[1, 0], [0, 1]])
P = np.array([2, 3])

result = is_point_in_lattice(L, P)
print(f"Result is: {result}")


Checks if a point P: [2 3] is in a given lattice L:
[[1 0]
 [0 2]]
Result is: False
--------------------------------------------------
Checks if a point P: [2 3] is in a given lattice L:
[[1 0]
 [0 1]]
Result is: True


## Πρόβλημα 2

Είσοδος: Πλέγματα L1 (πίνακας m x n1) και L2 (πίνακας m x n2).
Έξοδος: Βάση της τομής L1 ∩ L2.

1. Κατασκεύασε έναν νέο πίνακα L = [L1 | L2] (συνένωση των στηλών των L1 και L2).
2. Υπολόγισε το μηδενοχώρο του L.
4. Εξέτασε τους ακέραιους συνδυασμούς που ικανοποιούν και τις δύο βάσεις.
5. Επιστροφή: Οι στήλες του πίνακα που αντιστοιχούν σε βάσεις της τομής.

In [7]:
import numpy as np
from scipy.linalg import null_space

def lattice_intersection(L1, L2):
    """
    Computes the basis for the intersection of lattices L1 and L2.

    :param L1: numpy array (m x n1) representing the first lattice.
    :param L2: numpy array (m x n2) representing the second lattice.
    :return: numpy array (m x n) representing the basis for the intersection.
    """
    # Create [L1 | L2]
    L = np.hstack((L1, -L2))

    # Compute null space of L
    N = null_space(L)

    # Round for integers
    N = np.round(N).astype(int)

    # Compute intersection
    intersection_basis = np.dot(L1, N[:L1.shape[1], :])

    return intersection_basis


# Example
L1 = np.array([[1, 0], [0, 1]])
L2 = np.array([[2, 0], [0, 2]])

intersection = lattice_intersection(L1, L2)
print("A basis for the intersection L1 ∩ L2:")
print(intersection)

A basis for the intersection L1 ∩ L2:
[[1 0]
 [0 1]]


## Πρόβλημα 3

Input: a ∈ Z^m
Output: A basis of the lattice L

1. Let v=(v1,...,vn).
2. If n=2, then your basis is just (v2,−v1)
3. If n>2: Set the first vector of your basis to be b1=(−vn∗a1,−vn∗a2,...,−vn∗an−1,gcd(v1,...vn−1)), where the ai come from Bézout's Identity
4. Then complete your basis using the same algorithm with v=(v1,...vn−1).

In [11]:
from math import gcd
from functools import reduce
import numpy as np

def extended_euclidean(a, b):
    """Perform the Extended Euclidean Algorithm from geeksforgeeks"""
    # Base Case
    if a == 0 :
        return b,0,1

    gcd,x1,y1 = extended_euclidean(b%a, a)

    # Update x and y using results of recursive call
    x = y1 - (b//a) * x1
    y = x1

    return gcd,x,y

def bezout_coefficients(numbers):
    """
    Calculates Bézout coefficients for a list of integers.
    :param numbers: A list of integers.
    :return: A tuple containing the Bézout coefficients and the greatest common divisor.
    ~ used chatgpt
    """
    if len(numbers) < 2:
        raise ValueError("At least two integers are required.")

    # Pairwise computation of gcd and coefficients
    gcd_, x, y = extended_euclidean(numbers[0], numbers[1])
    coefficients = [x, y]

    for i in range(2, len(numbers)):
        gcd_, x, y = extended_euclidean(gcd_, numbers[i])
        coefficients = [coeff * x for coeff in coefficients] + [y]

    return coefficients, gcd_


def basis_verification(v,basis):
  """
  Verifies that v*basis=0
  The product of each row of the basis with given vector v must equal to zero
  """
  sum = 0
  for basis_vector in basis:
    sum +=np.dot(v, basis_vector)
  if(sum==0):
    return True
  else:
    return False


def lattice_basis(v):
  """Compute a basis of the lattice orthogonal to v.
  :param v: A list or numpy array representing the vector.
  :return: A numpry array of basis vectors for L = {x ∈ Z: v · x = 0}
  """
  v = np.array(v, dtype=int)
  basis = []
  n = 2
  while n<=len(v):
    v_temp = v[:n].copy()
    g = reduce(gcd, v_temp)
    if g != 1:
      v_temp //= g

    if n==2:
      # If n=2, then your basis is just (v2,−v1)
      basis_vector = np.array([v_temp[1], -v_temp[0]])

    else:
      # for n>2 basis_vec=(−vn∗a1,−vn∗a2,...,−vn∗an−1,gcd(v1,...vn−1))
      bezout_coeffs, g_last = bezout_coefficients(v_temp[:-1])
      basis_vector = [-v_temp[-1] * coeff for coeff in bezout_coeffs] + [g_last]
      basis_vector = np.array(basis_vector)


    # Add padding to the vector
    padding_length = len(v) - n
    if padding_length > 0:
      padding = np.zeros(padding_length, dtype=int)
      basis_vector = np.concatenate((basis_vector, padding))

    basis.append(basis_vector)
    n+=1

  basis = np.array(basis)
  return basis

# Example usage
v = [125, -75, 45, -27]
print("Input vector:",v)

# Compute the basis
basis = lattice_basis(v)
print("Basis L:")
print(basis)

# Verify the basis
print("Verification:", basis_verification(v,basis))


Input vector: [125, -75, 45, -27]
Basis L:
[[  -3   -5    0    0]
 [   9   18    5    0]
 [ -54 -108  -27    5]]
Verification: True


# Θέμα 8

## Βρες αν το διάνυσμα (43, 69, 95, 110) ανήκει στο πλέγμα

In [37]:
import numpy as np

# Ορισμός των πινάκων A και του διανύσματος v
A = np.array([[1, 2, 3, 4],
              [-1, 1, 2, 3],
              [4, 3, 2, 1],
              [0, 0, 1, 1]])

B = np.array([[5, 8, 11, 13],
              [4, 6, 8, 9],
              [8, 9, 10, 10],
              [4, 6, 9, 10]])

v = np.array([43, 69, 95, 110])

# Λύση για το σύστημα εξισώσεων A * x = v (γραμμικός συνδυασμός των γραμμών του A)
coefficients_A, resids, rank, s = np.linalg.lstsq(A.T, v, rcond=None)

# Έλεγχος αν το διάνυσμα ανήκει στο πλέγμα του A (αν τα υπόλοιπα είναι πολύ μικρά)
print("Συντελεστές για τις γραμμές του πίνακα A:", coefficients_A)
print("Υπόλοιπα για A:", resids)

# Λύση για το σύστημα εξισώσεων B * x = v (γραμμικός συνδυασμός των γραμμών του B)
coefficients_B, resids_b, rank_b, s_b = np.linalg.lstsq(B.T, v, rcond=None)

# Έλεγχος αν το διάνυσμα ανήκει στο πλέγμα του B (αν τα υπόλοιπα είναι πολύ μικρά)
print("\nΣυντελεστές για τις γραμμές του πίνακα B:", coefficients_B)
print("Υπόλοιπα για B:", resids_b)


Συντελεστές για τις γραμμές του πίνακα A: [14. 11. 10. 11.]
Υπόλοιπα για A: []

Συντελεστές για τις γραμμές του πίνακα B: [ 3.00000000e+00  9.00000000e+00 -1.00000000e+00 -4.17443857e-14]
Υπόλοιπα για B: []


## Βρες ορίζουσα πίνακα Α

In [38]:
import numpy as np

# Ορισμός του πίνακα A (ή B)
A = np.array([[1, 2, 3, 4],
              [-1, 1, 2, 3],
              [4, 3, 2, 1],
              [0, 0, 1, 1]])

# Υπολογισμός του όγκου του πλέγματος (ορίζουσα του πίνακα A)
volume_A = np.abs(np.linalg.det(A))

print("Ο όγκος του πλέγματος που παράγεται από τις γραμμές του πίνακα A είναι:", volume_A)

Ο όγκος του πλέγματος που παράγεται από τις γραμμές του πίνακα A είναι: 5.000000000000001


## Σημεία πλέγματος μέσα σε κύκλο και τετράγωνο

In [39]:
import numpy as np

# Ορισμός του πίνακα A
A = np.array([[1, 2, 3, 4],
              [-1, 1, 2, 3],
              [4, 3, 2, 1],
              [0, 0, 1, 1]])

B = np.array([[5, 8, 11, 13],
              [4, 6, 8, 9],
              [8, 9, 10, 10],
              [4, 6, 9, 10]])


# Συνάρτηση για να ελέγξουμε αν ένα διάνυσμα ανήκει στο πλέγμα
def is_in_lattice(vector, matrix):
    try:
        # Λύνουμε το σύστημα A * c = vector
        coefficients, resids, rank, s = np.linalg.lstsq(matrix.T, vector, rcond=None)
        # Ελέγχουμε αν τα υπόλοιπα (resids) είναι κοντά στο μηδέν
        if not np.allclose(np.dot(matrix.T, coefficients), vector):
            return False

        # Ελέγχουμε αν όλοι οι συντελεστές είναι ακέραιοι
        return np.allclose(coefficients, np.round(coefficients))
    except:
        return False

# Συνάρτηση για να υπολογίσουμε τα σημεία εντός του κύκλου
def points_in_circle(radius, matrix):
    count = 0
    for x in range(-radius, radius + 1):
        for y in range(-radius, radius + 1):
            if x**2 + y**2 < radius**2:  # Εντός του κύκλου
                vector = np.array([x, y, 0, 0])  # Μορφή (x, y, 0, 0)
                if is_in_lattice(vector, matrix):
                    count += 1
    return count

# Συνάρτηση για να υπολογίσουμε τα σημεία εντός του τετραγώνου
def points_in_square(side_length, matrix):
    count = 0
    half_side = side_length // 2
    for x in range(-half_side, half_side + 1):
        for y in range(-half_side, half_side + 1):
          if abs(x + y) + abs(x - y) < side_length:
            vector = np.array([x, y, 0, 0])  # Μορφή (x, y, 0, 0)
            if is_in_lattice(vector, matrix):
                count += 1
    return count

# Ρυθμίσεις
radius = 10  # Ακτίνα του κύκλου
side_length = 10  # Πλευρά του τετραγώνου

# Υπολογισμός
points_circle = points_in_circle(radius, B)
points_square = points_in_square(side_length, B)

print(f"Σημεία του πλέγματος μέσα στον κύκλο με ακτίνα 10: {points_circle}")
print(f"Σημεία του πλέγματος μέσα στο τετράγωνο με πλευρά 10: {points_square}")


Σημεία του πλέγματος μέσα στον κύκλο με ακτίνα 10: 61
Σημεία του πλέγματος μέσα στο τετράγωνο με πλευρά 10: 17


# Θέμα 3

In [13]:
###########################################################
#                                                         #
#                                                         #
#                 AUXILIARY FUNCTIONS                     #
#                         V1                              #
#                                                         #
#                                                         #
###########################################################
import numpy as np

def are_linearly_independent(vec1, vec2):
    """
    Checks if the 2 vectors are linearly independent.

    :param vec1: first vector (list or numpy array)
    :param vec2: second vector (list or numpy array)
    :return: True if they are linearly independent, False otherwise
    ~~ function created with chatgpt
    """
    # Μετατροπή σε numpy arrays για ευκολία
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)

    # Έλεγχος αν τα διανύσματα έχουν ίδιο μέγεθος
    if vec1.shape != vec2.shape:
        raise ValueError("Τα διανύσματα πρέπει να έχουν το ίδιο μέγεθος.")

    # Δημιουργία πίνακα από τα δύο διανύσματα
    matrix = np.column_stack((vec1, vec2))

    # Υπολογισμός ορίζουσας
    determinant = np.linalg.det(matrix)

    # Τα διανύσματα είναι γραμμικώς ανεξάρτητα αν η ορίζουσα δεν είναι 0
    return not np.isclose(determinant, 0)
'''
# Παράδειγμα χρήσης
vec1 = [1, 1]
vec2 = [3, 4]

print(are_linearly_independent(vec1, vec2))
'''

def calc_norm_squared(vec):
  """Calculates the Euclidean norm of a vector squared up.
  :param vec: A list or numpy array representing the vector.
  :return: The Euclidean norm of the vector squared up.
  """
  vector = np.array(vec)
  return np.sum(vector**2)


# prompt: create function to calculate the norm of a vector (gemini)
def calc_norm(vec):
  """Calculates the Euclidean norm of a vector.
  :param vec: A list or numpy array representing the vector.
  :return: The Euclidean norm of the vector.
  """
  vector = np.array(vec)
  return np.linalg.norm(vector)

def angle_between_vectors(vec1, vec2):
    """ Calculates the angle between two vectors in Z^2.
    :param v1: Το πρώτο διάνυσμα (λίστα ή numpy array)
    :param v2: Το δεύτερο διάνυσμα (λίστα ή numpy array)
    :return: Η γωνία σε μοίρες
    """
    # Transform into numpy arrays
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)

    # Calculation of dot product
    dot_product = np.dot(vec1, vec2)

    # Calculation of norms of the two vectors
    norm_v1 = np.linalg.norm(vec1)
    norm_v2 = np.linalg.norm(vec2)

    # Calculation of cos(θ)
    cos_theta = dot_product / (norm_v1 * norm_v2)

    # Check that cos(θ) in in [-1, 1] to avoid numeric errors
    cos_theta = np.clip(cos_theta, -1, 1)

    # Calculation in radians
    theta_radians = np.arccos(cos_theta)

    # Transformation in degrees
    theta_degrees = np.degrees(theta_radians)

    return theta_degrees
'''
# Example
v1 = [1, 2]
v2 = [3, 4]

angle = angle_between_vectors(v1, v2)
print(f"Η γωνία μεταξύ των διανυσμάτων είναι: {angle:.2f} μοίρες")
'''

# prompt: create function that generates a vector in Z^2 without limitations (Gemini)
def generate_vector_z2():
    """Generates a random vector in Z^2."""
    x = np.random.randint(-100, 101)
    y = np.random.randint(-100, 101)
    return np.array([x, y])


def create_vector_pairs():
  """ Finds two random linear independent vectors in Z^2
  :return: vec1, vec2"""

  found = False
  vec1 = generate_vector_z2()
  while not found:
    vec2 = generate_vector_z2()
    if are_linearly_independent(vec1,vec2):
      found = True

  return vec1, vec2


True


## Αλγόριθμος των Gauss-Lagrange

In [25]:
###########################################################
#                                                         #
#       MUST FIRST EXECUTE AUXILIARY FUNCTIONS-V1         #
#                                                         #
###########################################################
import math

def gauss_lagrange(vec1, vec2):
  """Outputs a Lagrange-Gauss reduced basis for the lattice L.
    :param vec1: The first vector (NumPy array).
    :param vec2: The second vector (NumPy array).
    :return: Basis (b1, b2) for L such that ||b_i|| = λ_i
    """

  b1 = calc_norm_squared(vec1)
  m = np.dot(vec1, vec2)/b1
  vec2 = vec2 - math.ceil(m-0.5)*vec1
  b2 = calc_norm_squared(vec2)

  while calc_norm(vec2) < calc_norm(vec1):
      # swap vector
      vec1, vec2 = vec2, vec1
      b1 = b2
      m = np.dot(vec1, vec2)/b1
      vec2 = vec2 - math.ceil(m-0.5)*vec1
      b2 = calc_norm_squared(vec2)

  return vec1, vec2

'''
# Requesting the vectors as input
while True:
    try:
        vec1_str = input("Enter the first vector (e.g., 1,2,3): ")
        vec1 = np.array([int(x) for x in vec1_str.split(',')])
        vec2_str = input("Enter the second vector (e.g., 4,5,6): ")
        vec2 = np.array([int(x) for x in vec2_str.split(',')])

        if not are_linearly_independent(vec1,vec2):
            print("Vectors must have the same dimension. Please enter again.")
            continue

        break
    except ValueError:
        print("Invalid input. Please enter numbers separated by commas.")
'''

vec1 =np.array([1,1])
vec2 =np.array([3,4])

GL_vec1, GL_vec2 = gauss_lagrange(vec1, vec2)
print("Gauss-Lagrange b1 =",GL_vec1)
print("Gauss-Lagrange b2 =",GL_vec2)

l1 = calc_norm(GL_vec1)
print("l1 =",l1)
print("-" * 50)

vec1 =np.array([-1,1])
vec2 =np.array([300,400])

GL_vec1, GL_vec2 = gauss_lagrange(vec1, vec2)
print("Gauss-Lagrange b1 =",GL_vec1)
print("Gauss-Lagrange b2 =",GL_vec2)

l1 = calc_norm(GL_vec1)
print("l1 =",l1)



Gauss-Lagrange b1 = [0 1]
Gauss-Lagrange b2 = [1 0]
l1 = 1.0
--------------------------------------------------
Gauss-Lagrange b1 = [-1  1]
Gauss-Lagrange b2 = [350 350]
l1 = 1.4142135623730951


## Gauss-Lagrange Experiment

Must first execute ***AUXILIARY FUNCTIONS-V1*** and ***gauss_lagrange*** algorithm

In [26]:
angle_list = []

for i in range(50):
  # find two linearly independent vectors in Z^2
  vec1, vec2 = create_vector_pairs()
  # calculate the Gauss-Lagrange vectors
  GL_vec1, GL_vec2 = gauss_lagrange(vec1, vec2)

  # Calulate the angle between the two vectors
  angle = angle_between_vectors(GL_vec1, GL_vec2)
  angle_list.append(angle)


angle_array = np.array(angle_list)
print("50 executions of Gauss-Lagrange:")
print(angle_array)
print("Average angle found:",angle_array.mean())



50 executions of Gauss-Lagrange:
[ 84.85176962 101.09372301  93.27645023  82.65534591  81.16151833
  98.00318599 106.26020471  85.66311904  72.83887832  97.14342759
  88.34542381 100.87166985  92.6025622  110.13902312  95.50547786
 102.43221532  93.00350583 108.24915623  92.40260947  80.68052157
  87.35745471  91.03535319  70.43932855  81.96834391 104.03624347
 101.23813329  92.55939246 108.31120003 105.88069671 111.27232848
  93.00191131 104.11444512 106.96271784  85.62162672  90.80920861
  73.02359014  88.50566641  78.594495    68.3566222   74.04258359
  81.81977012  98.64189871  89.85194922  90.86805145  72.95687429
  67.74135749  80.01139465 114.24048394  84.29934197  98.09589964]
Average angle found: 91.25676302447029


# Θέμα 4

## Αλγόριθμος Gram-Schmidt

In [27]:
import numpy as np

def gram_schmidt(basis):
    """
    Gram-Schmidt Algorithm to create an orthogonal basis.
    :param basis: a list of vecotrs (NumPy arrays) which create the basis B.
    :return orthogonal_basis: list of orthogonal vectors
    :return mu: Gram-Schmidt matrix (numpy array) with μ_ij cofacotrs

    ~ used chatgpt as a reference based on Alrgorithm 24 from Chapter 17
    Lattice Basis Reduction  of the book “Mathematics of Public Key Cryptography” by Steven Galbraithv
    """
    n = len(basis)
    # Initialize the orthogonal basis and the Gram-Schmidt matrix
    b_star = np.zeros_like(basis, dtype=float)
    mu = np.zeros((n, n), dtype=float)

    for i in range(n):
        b_star[i] = basis[i]
        for j in range(i):
          # Check for small norms to avoid division by zero added with chatgpt after debugging
          norm_b_star_j = np.dot(b_star[j], b_star[j])
          if np.abs(norm_b_star_j) < 1e-10:
            raise ValueError(f"Basis vector {j} is too small or degenerate.")
          mu[i, j] = np.dot(basis[i], b_star[j]) / np.dot(b_star[j], b_star[j])
          b_star[i] -= mu[i, j] * b_star[j]

    return b_star, mu

# Παράδειγμα χρήσης
basis = [
    np.array([1.0, 1.0]),
    np.array([1.0, 0.0])
]

orthogonal_basis, mu = gram_schmidt(basis)

# Εκτύπωση αποτελεσμάτων
print("Ορθογώνια βάση:")
for vec in orthogonal_basis:
    print(vec)

print("\nGram-Schmidt πίνακας (μ):")
print(mu)


Ορθογώνια βάση:
[1. 1.]
[ 0.5 -0.5]

Gram-Schmidt πίνακας (μ):
[[0.  0. ]
 [0.5 0. ]]


## Αλγόριθμος LLL

In [28]:
###########################################################
#                                                         #
#       MUST FIRST EXECUTE gram_schmidt()                 #
#                                                         #
###########################################################
import math
import numpy as np

def lll_algorithm(basis, delta):
  """
    LLL algorithm with Euclidean norm
    :param orthogonal_basis: a list of vecotrs (NumPy arrays) which the basis B.
    :param delta: typically choose δ = 3/4
    :return basis: LLL reduced basis b1, . . . , bn (numpy array)
  """
  n = len(basis)
  orthogonal_basis, mu = gram_schmidt(basis)
  # Compute Bi = <{b_i}*,{b_i}*> = ||{b_i}*||^2
  B = np.array([np.dot(orthogonal_basis[i], orthogonal_basis[i]) for i in range(n)])

  k=1
  while k < n:
    # Perform size reduction
    for j in range(k - 1, -1, -1):
      q = round(mu[k, j])
      basis[k] -= q * basis[j]
      orthogonal_basis, mu = gram_schmidt(basis)
      #B = np.array([np.dot(orthogonal_basis[i], orthogonal_basis[i]) for i in range(n)])

    # Check Lov´asz condition
    if B[k] < (delta-mu[k,k-1]**2)*B[k-1]:
        # Swap basis[k] and basis[k-1]
        basis[k], basis[k-1] = basis[k-1], basis[k].copy()
        orthogonal_basis, mu = gram_schmidt(basis)
        B = np.array([np.dot(orthogonal_basis[i], orthogonal_basis[i]) for i in range(n)])
        k = max(1, k - 1)
    else:
      k+=1
  return basis


delta = 0.99
basis = np.array([[10,1], [1, 9]])
lll_reduced_basis = lll_algorithm(basis.copy(),delta)
print("LLL Reduced Basis:")
print(lll_reduced_basis)

LLL Reduced Basis:
[[ 1  9]
 [10  1]]


In [29]:
# Examples
delta = 0.99
basis = np.array([[1, 1, 1], [1, 0, 2], [0, 1, 1]])
lll_reduced_basis = lll_algorithm(basis.copy(),delta)
print("LLL Reduced Basis:")
print(lll_reduced_basis)

basis = np.array([[10,1], [1, 9]])
lll_reduced_basis = lll_algorithm(basis.copy(),delta)
print("LLL Reduced Basis:")
print(lll_reduced_basis)

basis = np.array([[1, -1, 3], [1, 0, 5], [1, 2, 6]])
lll_reduced_basis = lll_algorithm(basis.copy(),delta)
print("LLL Reduced Basis:")
print(lll_reduced_basis)

basis = np.array([[-36,32], [-32, -27]])
lll_reduced_basis = lll_algorithm(basis.copy(),delta)
print("LLL Reduced Basis:")
print(lll_reduced_basis)

basis = np.array([[-1, -1, -2, -4, -1], [-2, -2, 4, -4, 2], [4, 2, -1, 1, 4], [1, -1, 1, -1, -1],[-1, 1, -1, 1, -30]])
lll_reduced_basis = lll_algorithm(basis.copy(),delta)
print("LLL Reduced Basis:")
print(lll_reduced_basis)

basis = np.array([[42, -36, -5], [-13, 9, 30]])
lll_reduced_basis = lll_algorithm(basis.copy(),delta)
print("LLL Reduced Basis:")
print(lll_reduced_basis)

LLL Reduced Basis:
[[-1  0  0]
 [ 0 -1  1]
 [ 0  1  1]]
LLL Reduced Basis:
[[ 1  9]
 [10  1]]
LLL Reduced Basis:
[[ 1 -1  0]
 [-1  0  1]
 [ 1  1  1]]
LLL Reduced Basis:
[[-32 -27]
 [-36  32]]
LLL Reduced Basis:
[[ 1 -1  1 -1 -1]
 [-2  0 -3 -3  0]
 [ 5  1  0  0  3]
 [-3 -1  3 -3  3]
 [-1  7  2 -2 -6]]
LLL Reduced Basis:
[[-13   9  30]
 [ 29 -27  25]]


## Πρώτο διάνυσμα που δίνει ο LLL έχει μεγαλύτερο μήκος από το δεύτερο διάνυσμα

In [30]:
###########################################################
#                                                         #
#       MUST FIRST EXECUTE AUXILIARY FUNCTIONS-V1         #
#                 and  lll_algorithm                      #
###########################################################
def generate_random_lattice(dim, max_val=10000):
    """
    Generates a random basis for an dim-dimensional lattice.
    :parm dim: dimention of the lattice.
    :parm max_val (int): The maximum absolute value of the elements in the basis matrix.
    :returns a numpy array representing the lattice basis.
    ~ created with chatgpt
    """
    while True:
      # Generate a random n x n matrix with integers
      basis = np.random.randint(-max_val, max_val + 1, size=(dim, dim))

      # Check if the rows are linearly independent (determinant is non-zero)
      if np.linalg.det(basis) != 0:
        return basis

def check_lll_norm_condition(max_iterations=100):
  """
  Checks random lattices (up to max_iterations) until it finds one where the
  norm of the first vector of the LLL-reduced lattice is greater than the second.
  :parm max_iterations: The maximum number of iterations to perform.
  :returns a tuple containing the original base, the LLL-reduced basis and the norms if the condition is met,
    None otherwise.

  ~ used Gemini as reference for naming
  """

  for _ in range(max_iterations):
    basis = generate_random_lattice(4)

    # Apply the LLL algorithm
    lll_basis = lll_algorithm(basis.copy(), delta=0.75)

    # Check the condition
    norm_first = calc_norm(lll_basis[0])
    norm_second = calc_norm(lll_basis[1])

    if norm_first > norm_second:
        return  basis, lll_basis, (norm_first, norm_second)

  return None


result = check_lll_norm_condition()

if result:
    basis, lll_basis, norms = result
    print("Original basis found satisfying the condition:")
    print(basis)
    print("LLL-reduced basis found satisfying the condition:")
    print(lll_basis)
    print("Norms:", norms)
else:
    print("No LLL-reduced basis found satisfying the condition within the given iterations.")

Original basis found satisfying the condition:
[[-5576 -9677 -4263  4281]
 [-9604 -7497 -3527 -9617]
 [ 9709 -4638 -2658  1833]
 [-1044  7527 -6695 -9673]]
LLL-reduced basis found satisfying the condition:
[[-5576 -9677 -4263  4281]
 [ 9709 -4638 -2658  1833]
 [ 2984  5347 -7431  4225]
 [-1044  7527 -6695 -9673]]
Norms: (12697.883091287304, 11233.90306171457)


## Τυχαία εκτέλεση LLL και Gauss-Lagrange σε πλέγματα βαθμίδας 2

In [31]:
###########################################################
#                                                         #
#       MUST FIRST EXECUTE AUXILIARY FUNCTIONS-V1,        #
#           lll_algorithm and gauss_lagrange              #
###########################################################
def random_executions(iterations):
  """
  Checks <iterations> random lattices
  :parm iterations: The number of iterations to perform.
  """
  counter = 0
  for _ in range(iterations):
    # Generate two random linearly independent vectors in Z^2
    vec1, vec2 = create_vector_pairs()
    basis = np.array([vec1, vec2])

    # Apply the LLL algorithm
    lll_basis = lll_algorithm(basis.copy(), delta=0.75)

    # Apply the Gauss-Lagrance algorithm
    GL_vec1, GL_vec2 = gauss_lagrange(vec1, vec2)
    gauss_basis = np.array([GL_vec1, GL_vec2])

    """
    print("Original basis:")
    print(basis)
    print("LLL-reduced basis:")
    print(lll_basis)
    print("Gauss-Lagrange-reduced basis:")
    print(gauss_basis)
    """
    if  np.array_equal(lll_basis, gauss_basis):
      #print("The reduced bases are the same")
      counter += 1
    #else:
      #print("The reduced bases are NOT the same")
    #print("----------------------------------------")
  print(counter, "out of ",iterations, "exapmles were the same")

  return None


run = random_executions(100)

92 out of  100 exapmles were the same


# Θέμα 10
## Αλγόριθμος του Bambai

In [40]:
###########################################################
#                                                         #
#       MUST FIRST EXECUTE AUXILIARY FUNCTIONS-V1,        #
#           lll_algorithm and gram_schmidt                #
###########################################################
def bambai_algorithm(basis, t):
  """
  Babai’s nearest plane algorithm

  :param basis: Basis matrix (n x m) (NumPy arrays) describing the lattice.
  :param t: Target vector (NumPy arrays), m-dimensional
  :return x: The closest lattice point to t.
  """
  n = len(basis)
  # Apply the LLL algorithm
  lll_basis = lll_algorithm(basis.copy(), delta=0.75)

  # Apply Gram-Schmidt Algorithm to create an orthogonal basis
  orthogonal_basis, mu = gram_schmidt(lll_basis.copy())

  b = t.copy()
  for j in range(n - 1, -1, -1):
    c_j = round(np.dot(b,orthogonal_basis[j])/np.dot(orthogonal_basis[j],orthogonal_basis[j]))
    b -= c_j* lll_basis[j]
  return t-b


# Examples
#B = np.array([[1, 0], [0, 1]])
#t = np.array([2.7, 3.4])
#closest_point = bambai_algorithm(B, t)
#print("The closest lattice point is:", closest_point)

B = np.array([[137, 312], [215, -187]])
t = np.array([53172, 81743])
closest_point = bambai_algorithm(B, t)
print("The closest lattice point is:", closest_point)


The closest lattice point is: [53159 81818]


# Θέμα 11
## Αλγόριθμος KFP

In [42]:
###########################################################
#                                                         #
#       MUST FIRST EXECUTE AUXILIARY FUNCTIONS-V1,        #
#                 and gram_schmidt                        #
###########################################################

def kfp_enumeration(basis, R):
    """
    KFP enumeration algorithm returns all vectors of L which are smaller than R

    :param basis: Basis matrix (n x n) (NumPy arrays) describing the lattice.
    :param R: possitive number constraint for the norm
    :return:  A set of lattice vectors satisfying ||x|| <= R
    """
    n = len(basis)
    # compute {μ_{ij}}
    b_star, mu = gram_schmidt(basis.copy())
    # Compute Bi = ||{b_i}*||^2
    B = np.array([np.dot(b_star[i], b_star[i]) for i in range(n)])

    x = np.zeros(n, dtype=int)      # Initialize x vector
    c = np.zeros(n, dtype=float)    # Initialize c vector
    l = np.zeros(n, dtype=float)    # Initialize l vector
    sumli = 0
    S = set()                       # Set to store lattice vectors
    i = 0

    while i < n:
        # Update c[i] as the summation of μ[j, i] * x[j] for j=i+1 to n
        c[i] = -sum(x[j] * mu[j][i] for j in range(i+1, n))

        l[i] = B[i] * ((x[i] - c[i])**2)
        sumli = sum(l[i:n])

        if sumli <= R**2:
            if i == 0:
                S.add(tuple(sum(x[j] * basis[j] for j in range(n))))
                x[0] += 1
            else:
                i -= 1
                summation = sum(mu[j, i] * x[j] for j in range(i + 1, n))
                # xi ← left part of the interval Ii
                x[i] = math.ceil(-summation - np.sqrt((R**2 - sum(l[i+1:n])) / B[i]))
        else:
            i += 1
            if i < n:
              x[i] += 1
    return S

# Example
basis = np.array([[3, 6, 13], [11, 3, 15], [12, 12, 0]])
R = np.sqrt(214)

result = kfp_enumeration(basis, R)
print("Lattice vectors satisfying ||x|| <= R:")
for vector in result:
    print(vector)


Lattice vectors satisfying ||x|| <= R:
(8, -3, 2)
(0, 0, 0)
(3, 6, 13)


In [ ]:
# Παράδειγμα χρήσης
basis = np.array([[-1, -1, -2, -4, -1], [-2, -2, 4, -4, 2], [4, 2, -1, 1, 4], [1, -1, 1, -1, -1], [-1, 1, -1, 1, 30]])
R = np.sqrt(40)

result = kfp_enumeration(basis, R)
print("Vectors satisfying the condition:")
for vector in result:
    print(vector)

Vectors satisfying the condition:
(5, 1, 0, 0, 3)
(3, 1, -3, -3, 3)
(1, -1, 1, -1, -1)
(0, -2, -1, -5, -2)
(2, 0, 3, 3, 0)
(3, -1, 4, 2, -1)
(-1, -1, -2, -4, -1)
(4, 2, -1, 1, 4)
(0, 0, 0, 0, 0)
(4, 0, -2, 2, -4)
(2, -2, 2, -2, -2)
(4, 0, -2, -4, 2)
(3, 1, -3, 3, -3)


# Θέμα 12
## ́Αλγόριθμος Schnorr-Euchner

In [ ]:
def schnorr_euchner_algorithm(basis, R):
  """
  KFP enumeration algorithm returns all vectors of L which are smaller than R

    :param basis: Basis matrix (n x n) (NumPy arrays) describing the lattice.
    :param R: possitive number constraint for the norm
    :return:  A set of lattice vectors satisfying ||x|| <= R
  """
  n = len(basis)
  # compute {μ_{ij}}
  b_star, mu = gram_schmidt(basis.copy())
  # Compute Bi = ||{b_i}*||^2
  B = np.array([np.dot(b_star[i], b_star[i]) for i in range(n)])

  x = np.zeros(n, dtype=int)      # Initialize x vector
  c = np.zeros(n, dtype=float)    # Initialize c vector
  l = np.zeros(n, dtype=float)    # Initialize l vector

  # Δx ← (1, 0n−1)
  delta_x = np.zeros(n)
  delta_x[0] = 1

  # Δ2x ← (1, (−1)n−1)
  delta_2x = np.ones(n)
  delta_2x[1:] = -1

  sumli = 0
  S = set()                       # Set to store lattice vectors
  i = 0

  while i < n:
    c[i] = -sum(x[j]*mu[j][i] for j in range(i+1, n))
    l[i] = B[i]*((x[i]-c[i])**2)
    sumli = sum(l[i:n])
    if sumli <= R**2 and i == 0:
      S.add(tuple(sum(x[j]*basis[j] for j in range(n))))
    if sumli <= R**2 and i > 0:
      i -= 1
      c[i] = -sum(x[j]*mu[j][i] for j in range(i+1, n)) # center of the interval In+1−i
      x[i] = round(c[i])
      delta_x[i] = 0
      if c[i] < x[i]:
        delta_2x[i] = 1
      else:
        delta_2x[i] = -1
    elif i ==n:         # i.e. if sumli > R2 and i = n
      break
    else:               # i.e. if sumli > R2 or i = 1
      i +=1
      if i < n:
        delta_2x[i] = -delta_2x[i]
        delta_x[i] = -delta_x[i] + delta_2x[i]
        x[i] = x[i] + delta_x[i]
  return S


# Example
basis = np.array([[3, 6, 13], [11, 3, 15], [12, 12, 0]])
R = np.sqrt(214)

result = schnorr_euchner_algorithm(basis, R)
print("Lattice vectors satisfying ||x|| <= R:")
for vector in result:
    print(vector)


Lattice vectors satisfying ||x|| <= R:
(8, -3, 2)
(0, 0, 0)
(-8, 3, -2)


In [ ]:
# Παράδειγμα χρήσης
basis = np.array([[-1, -1, -2, -4, -1], [-2, -2, 4, -4, 2], [4, 2, -1, 1, 4], [1, -1, 1, -1, -1], [-1, 1, -1, 1, 30]])
R = np.sqrt(40)

result = schnorr_euchner_algorithm(basis, R)
print("Vectors satisfying the condition:")
for vector in result:
    print(vector)

Vectors satisfying the condition:
(-1, 1, -1, 1, 1)
(-3, -1, 3, 3, -3)
(5, 1, 0, 0, 3)
(1, -1, 1, -1, -1)
(3, 1, -3, -3, 3)
(-2, 2, -2, 2, 2)
(-4, 0, 2, -2, 4)
(-5, -1, 0, 0, -3)
(-3, -1, 3, -3, 3)
(0, 0, 0, 0, 0)
(4, 0, -2, 2, -4)
(2, -2, 2, -2, -2)
(3, 1, -3, 3, -3)


## Εnumeration with prunning και prunning vector

In [43]:
###########################################################
#                                                         #
#       MUST FIRST EXECUTE AUXILIARY FUNCTIONS-V1,        #
#                 and gram_schmidt                        #
###########################################################

import math

def compute_pruning_vector(n):
    """
    Computes the pruning vector a_i = min(1, sqrt(1.05^(i/n))).
        :param n (int): Dimension of the lattice.
        :return: list of float: Pruning vector.
    """
    return [min(1, math.sqrt(1.05**(i/n))) for i in range(n)]

def schnorr_euchner_algorithm_with_prunning(basis, R):
  """
  KFP enumeration algorithm returns all vectors of L which are smaller than R

    :param basis: Basis matrix (n x n) (NumPy arrays) describing the lattice.
    :param R: possitive number constraint for the norm
    :return:  A set of lattice vectors satisfying ||x|| <= R
  """
  n = len(basis)
  # compute {μ_{ij}}
  b_star, mu = gram_schmidt(basis.copy())
  # Compute Bi = ||{b_i}*||^2
  B = np.array([np.dot(b_star[i], b_star[i]) for i in range(n)])

  x = np.zeros(n, dtype=int)      # Initialize x vector
  c = np.zeros(n)                 # Initialize c vector
  l = np.zeros(n)                 # Initialize l vector

  # Δx ← (1, 0n−1)
  delta_x = np.zeros(n)
  delta_x[0] = 1

  # Δ2x ← (1, (−1)n−1)
  delta_2x = np.ones(n)
  delta_2x[1:] = -1

  sumli = 0
  S = set()                       # Set to store lattice vectors
  i = 0

  # compute prunning vvector
  a = compute_pruning_vector(n)

  while i < n:
    c[i] = -sum(x[j]*mu[j][i] for j in range(i+1, n))
    l[i] = B[i]*((x[i]-c[i])**2)
    sumli = sum(l[i:n])
    if sumli <= (a[n-1-i]*R)**2 and i == 0:
      S.add(tuple(sum(x[j]*basis[j] for j in range(n))))
    if sumli <= (a[n-1-i]*R)**2 and i > 0:
      i -= 1
      c[i] = -sum(x[j]*mu[j][i] for j in range(i+1, n)) # center of the interval In+1−i
      x[i] = round(c[i])
      delta_x[i] = 0
      if c[i] < x[i]:
        delta_2x[i] = 1
      else:
        delta_2x[i] = -1
    elif i ==n:         # i.e. if sumli > R2 and i = n
      break
    else:               # i.e. if sumli > R2 or i = 1
      i +=1
      if i < n:
        delta_2x[i] = -delta_2x[i]
        delta_x[i] = -delta_x[i] + delta_2x[i]
        x[i] = x[i] + delta_x[i]
  return S


# Example
basis = np.array([[3, 6, 13], [11, 3, 15], [12, 12, 0]])
R = np.sqrt(214)

result = schnorr_euchner_algorithm_with_prunning(basis, R)
print("Lattice vectors satisfying ||x|| <= R:")
for vector in result:
    print(vector)


Lattice vectors satisfying ||x|| <= R:
(8, -3, 2)
(0, 0, 0)
(-8, 3, -2)


# Θέμα 4 - vol.2

In [35]:
###########################################################
#                                                         #
#       MUST FIRST EXECUTE AUXILIARY FUNCTIONS-V1,        #
#           lll_algorithm  and  kfp_enumeration           #
###########################################################
import numpy as np
import random

iterations = 1

def is_lll_first_vector_lambda1(basis, delta=0.75):
    """
    Check if the first vector of the LLL-reduced basis is λ1(L).

      :param basis: Input lattice basis (numpy array).
      :param delta: Delta parameter for LLL (default 0.75).
      :return: True if the first vector is λ1(L), otherwise False.
    """
    # Compute the LLL-reduced basis
    lll_reduced_basis = lll_algorithm(basis.copy(),delta)
    lll_shortest_norm = np.linalg.norm(lll_reduced_basis[0])
    #print("LLL Reduced Basis:")
    #print(lll_reduced_basis)
    #print("LLL First Vector Norm:", lll_shortest_norm)

    # Use KFP to find λ1(L)
    R = lll_shortest_norm + 0.01  # Slightly above the LLL first vector's norm
    enumerated_vectors  = kfp_enumeration(basis, R)

    # Find the shortest norm among enumerated vectors ~ used chatgpt
    min_enumerated_norm = float('inf')
    for vec in enumerated_vectors:
      norm = np.linalg.norm(vec)
      if norm > 0:  # Exclude the zero vector
        min_enumerated_norm = min(min_enumerated_norm, norm)

    # Compare norms
    return np.isclose(lll_shortest_norm, min_enumerated_norm), lll_reduced_basis[0], min_enumerated_norm


In [ ]:
# Example
true_counter = 0
iterations = 50
for _ in range(iterations):
  lattice = generate_random_lattice(4)
  result, lll_vector, lambda1 = is_lll_first_vector_lambda1(lattice)
  if result:
    true_counter += 1
  #print("Lattice:")
  #print(lattice)
  #print("LLL First Vector:", lll_vector)
  print("λ1(L) from Enumeration:", lambda1)
  print("Is LLL first vector λ1(L)?", result)
  print("-" * 50)
print(true_counter, "cases out of ", iterations, "were correct")

In [36]:
import numpy as np

# Example to find approximation
counter_list = []
iterations = 50
for _ in range(10):
  true_counter = 0
  for _ in range(iterations):
    # Generate random Lattice of max dimention 4
    lattice = generate_random_lattice(random.randint(2, 4))
    result, lll_vector, lambda1 = is_lll_first_vector_lambda1(lattice)
    if result:
      true_counter += 1
  counter_list.append(true_counter)
  print(true_counter, "cases out of ", iterations, "were correct")
counter_list=np.array(counter_list)
#print(counter_list)
print("Average:", np.mean(counter_list))


44 cases out of  50 were correct
41 cases out of  50 were correct
41 cases out of  50 were correct
43 cases out of  50 were correct
42 cases out of  50 were correct
40 cases out of  50 were correct
43 cases out of  50 were correct
41 cases out of  50 were correct
45 cases out of  50 were correct
41 cases out of  50 were correct
Average: 42.1


# Θέμα 6

In [ ]:
import numpy as np
from sympy import Poly, ceiling, roots
from sympy.abc import x
from scipy.linalg import lu

def univariate_coppersmith(f_coeffs, N, epsilon):
    """
    Implements the Univariate Coppersmith method in Python.

    Parameters:
    - f_coeffs: Coefficients of the polynomial (highest degree first).
    - N: The modulus.
    - epsilon: Small positive value for the bound X.

    Returns:
    - Roots of the polynomial f modulo N.
    """
    f = Poly(f_coeffs, x, domain='ZZ')
    d = f.degree()

    # Compute h using Galbraith's formula
    h = ceiling(1 / (d * epsilon))
    print(f"h = {h}")
    if h == 1:
        raise ValueError("Choose a new value for epsilon.")

    # Compute bound X
    X = ceiling(0.5 * N**(1 / d - epsilon))
    print(f"X = {X}")

    # Compute lattice rank (n)
    n = d * h
    print(f"Rank of lattice (n) = {n}")

    # Construct the lattice basis matrix
    L = []
    #scaled_x = x * X  # Scale the variable x by X
    for j in range(h):
      for i in range(d):
        #term = N**(h - 1 - j) * (scaled_x)**i * f(scaled_x)**j # removed scaled_x
        term = N**(h - 1 - j) * (x * X)**i * f.subs({x: x * X})**j # Using .subs to avoid type mismatch.
        # Changed to ensure that when evaluating the polynomial f, instead of
        # calling f with the scaled_x (which introduced symbolic x in the coefficients),
        # we explicitly substitute x with x * X using .subs({x: x * X})
        # This keeps the coefficients as integers.
        term_poly = Poly(term.as_expr().expand(), x, domain='ZZ')
        row = [term_poly.coeff_monomial(x**k) if k <= term_poly.degree() else 0 for k in range(n)]
        L.append(row)

    # Convert lattice basis to numpy array
    L = np.array(L, dtype=object)

    # Perform LLL reduction using scipy's LU decomposition (approximation)
    P, L_, U = lu(L.astype(np.float64))
    reduced_basis = U.astype(object)

    # Construct polynomial from the first basis vector
    g_coeffs = [reduced_basis[0, i] / (X**i) for i in range(n)]
    g = Poly(g_coeffs, x, domain='QQ')

    # Find roots of the polynomial modulo N
    g_roots = roots(g, multiple=True)

    return [int(r) for r in g_roots if isinstance(r, int) and 0 <= r < N]

# Example usage:
# Coefficients of the polynomial (highest degree first)
f_coeffs = [987654321987654321, 1234567890123456789, 1942528644709637042, 1]
N = (2**30 + 3) * (2**32 + 15)
epsilon = 0.09

roots = univariate_coppersmith(f_coeffs, N, epsilon)
print("Roots:", roots)


h = 4
X = 17399
Rank of lattice (n) = 12
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Roots: []


In [ ]:
import numpy as np
from sympy import Poly, ceiling, symbols, Matrix
from sympy.abc import x
#!pip install fpylll


def univariate_coppersmith(f_coeffs, N, epsilon):
    """
    Implements the Univariate Coppersmith method in Python.

    Parameters:
    - f_coeffs: Coefficients of the polynomial (highest degree first).
    - N: The modulus.
    - epsilon: Small positive value for the bound X.

    Returns:
    - Roots of the polynomial f modulo N.
    """
    f = Poly(f_coeffs, x, domain='ZZ')
    d = f.degree()

    # Compute h using Galbraith's formula
    h = ceiling(1 / (d * epsilon))
    print(f"h = {h}")
    if h == 1:
        raise ValueError("Choose a new value for epsilon.")

    # Compute bound X
    X = ceiling(0.5 * N**(1 / d - epsilon))
    print(f"X = {X}")

    # Compute lattice rank (n)
    n = d * h
    print(f"Rank of lattice (n) = {n}")

    # Construct the lattice basis matrix
    L = []
    for j in range(h):
        for i in range(d):
            term = N**(h - 1 - j) * (x * X)**i * f.subs({x: x * X})**j
            term_poly = Poly(term.as_expr().expand(), x, domain='ZZ')
            row = [term_poly.coeff_monomial(x**k) if k <= term_poly.degree() else 0 for k in range(n)]
            L.append(row)

    # Convert lattice basis to fpylll IntegerMatrix
    L_matrix = np.array(L)

    # Perform LLL reduction
    lll_reduced = lll_algorithm(L_matrix,0.99)

    # Extract the first row of the reduced basis
    reduced_basis = np.array(lll_reduced, dtype=object)
    g_coeffs = [reduced_basis[0, i] // (X**i) for i in range(n)]
    g = Poly(g_coeffs, x, domain='ZZ')

    # Find roots of the polynomial modulo N
    g_roots = g.nroots()  # Numerical roots

    return [int(round(r)) for r in g_roots if r.is_real and 0 <= round(r) < N]

# Example usage:
# Coefficients of the polynomial (highest degree first)
f_coeffs = [1, 1234567890123456789, 987654321987654321, 1942528644709637042]
N = (2**30 + 3) * (2**32 + 15)
epsilon = 0.09

roots = univariate_coppersmith(f_coeffs, N, epsilon)
print("Roots:", roots)


h = 4
X = 17399
Rank of lattice (n) = 12


NoConvergence: convergence to root failed; try n < 15 or maxsteps > 50

# Θέμα 5

In [ ]:
###########################################################
#                                                         #
#                                                         #
#               YOU MUST RUN θΕΜΑ 4 FIRST                 #
#                  FOR LLL ALGORITHM                      #
#                                                         #
#                                                         #
###########################################################


import numpy as np
import random, operator, math
import sys

# Dot product function
def dot_product(a, b):
    return sum(map(operator.mul, a, b))

# Generate random binary array
def rand_bin_array(K, N):  # K zeros, N-K ones
    arr = np.array([0] * K + [1] * (N - K))
    np.random.shuffle(arr)
    return arr

# Function to create a knapsack problem
def find(n,d,hamming):                                            #balanced hamming weight
    if n%2==1:
        return "enter an even integer"
    a =  [random.randint(1,math.floor(2**((2-d)*n))) for _ in range(n)]
    density=float(len(a)/math.log(max(a),2));
    solution = rand_bin_array(n-hamming,n)
    a0 = dot_product(solution,a);
    aold=a;
    a0old=a0;
    return a,a0,density,sum(solution),len(solution),solution

# Knapsack Lemma Lattice Construction
def construct_lattice(a, a0, N, H):
    n = len(a)
    B = np.zeros((n + 1, n + 3), dtype=int)

    # Fill the n-rows of the lattice matrix
    for i in range(n):
      B[i, i] = 2
      B[i, -1] = N
      B[i, -3] = a[i]*N

    # Fill the last row of the lattice matrix
    for i in range(n + 3):
      B[n, i] = 1

    B[n, -1] = H*N
    B[n, -3] = a0*N

    return B

def generate_b(n, x, sign):
    """
    Δημιουργεί το διάνυσμα b μήκους n+3 με τις εξής προϋποθέσεις:
    - |b_(n+2)| = 1
    - Για 14 από τα πρώτα n στοιχεία: b_i ≠ b_(n+2)
    - Για τα υπόλοιπα από τα πρώτα n στοιχεία: b_i = b_(n+2)
    - b_(n+1) = 0
    - b_(n+3) = 0

    Parameters:
        n (int)   : Το μήκος του διανύσματος b είναι n+3.
        x         : Διάνυσμα λύσης

    Returns:
        numpy.ndarray: Το διάνυσμα b με τις ιδιότητες που ορίζονται στο πρόβλημα.
    """
    # Δημιουργία του διανύσματος b με μηδενικά στοιχεία αρχικά
    b = np.zeros(n + 3)

    # Επιλογή τυχαίας τιμής για b_(n+2)
    #b[-2] = np.random.choice([-1, 1])
    b[-2] = sign

    for i in range(n):
      if x[i] == 1:
        # αν το i είναι μέρος της λύσης τότε πρέπει το bi!=bn-2
        b[i] = -b[-2]
      else:
        b[i] = b[-2]

    return b


def find_vector_b(reduced_basis, n):
    """
    Find a vector b in the lattice satisfying the given conditions.

    Args:
        reduced_basis (numpy.ndarray): The LLL-reduced lattice basis matrix.
        n (int): The dimension of the knapsack problem.

    Returns:
        numpy.ndarray: The vector b satisfying the conditions.
    """
    rows, cols = reduced_basis.shape
    for combination in range(1, 2**rows):  # Try all combinations of rows
        # Generate coefficients for the linear combination
        coefficients = [(combination >> i) & 1 for i in range(rows)]
        b = sum(coeff * row for coeff, row in zip(coefficients, reduced_basis))

        # Check conditions
        if all(abs(b[i]) == 1 for i in range(n)):  # |b1| = |b2| = ... = |bn| = 1
            if abs(b[n+1]) == 1:  # |bn+2| = 1
                if b[n] == 0 and b[n+2] == 0:  # bn+1 = 0, bn+3 = 0
                    return b

    raise ValueError("No valid vector b found in the lattice.")


def check_vector_b(b):
    """
    Checks if Knapsack Lemma holds for a given vector.
    :param b (numpy.ndarray): The vector to check.
    :returns bool: True if the vector satisfies the conditions, False otherwise.
    """
    n = len(b) - 3
    if not all(abs(b_i) == 1 for b_i in b[:n]):
      return False
    if abs(b[n + 1]) != 1 :
      return False
    if b[n] != 0 or b[n + 2] != 0:
        return False
    return True


# Solve knapsack with LLL
def solve_knapsack_with_lll(n, d, hamming):
    # Step 1: Create knapsack problem
    a, a0, density, solution_weight, _, solution = find(n, d, hamming)
    N = math.ceil(np.sqrt(n)) + 1
    print("################################# STEP 1 - Create knapsack problem #################################")
    print("Knapsack Problem:")
    print("a:", a)
    print("a0:", a0)
    print("Density:", density)
    print("Solution Weight:", solution_weight)
    np.set_printoptions(threshold=sys.maxsize)
    print("Solution:", solution)
    np.set_printoptions(threshold = False)
    print("N:", N)

    # Step 2: Construct lattice
    print("################################# STEP 2 - Construct lattice #################################")
    lattice = construct_lattice(a, a0, N, hamming)
    print("Lattice:")
    print(lattice)

    # Step 3: Apply LLL reduction
    print("################################# STEP 3 - Apply LLL reduction #################################")
    reduced_lattice = lll_algorithm(lattice.copy(),0.99)
    print("Reduced Lattice:")
    print(reduced_lattice)


    # Step 4: Recover solution
    print("################################# STEP 4 - Extract and verify solution #################################")
    for vector in reduced_lattice:
      lll_solution = 0
      for i in range(n):
        lll_solution += a[i]*abs(vector[i]-vector[-2])/2
      #print("LLL Solution:", lll_solution)
      #print("Original  S :", a0)
      #print("LLL Solution 2:", lll_solution2)
      if(lll_solution == a0):
        return lll_solution, vector, a0
    return None, None, a0


# Parameters
n = 30
d = 1
hamming = 15

# Solve the knapsack problem with LLL
recovered_solution, vector, true_solution = solve_knapsack_with_lll(n, d, hamming)

# Output results
print("################################# RESULTS #################################")
if recovered_solution is not None:
    print("Recovered LLL solution:", recovered_solution)
    print("True solution:", true_solution)
    print("Match:", recovered_solution == true_solution)
    np.set_printoptions(threshold=sys.maxsize)
    print("Vector:", vector)
    np.set_printoptions(threshold = False)
    if check_vector_b(vector):
      print("Vector satisfies the Knapsack Lemma.")
    else:
      print("Vector does not satisfy the Knapsack Lemma.")
else:
    print("No solution found.")

################################# STEP 1 - Create knapsack problem #################################
Knapsack Problem:
a: [654966885, 900724656, 444390706, 771075754, 571803029, 882008365, 680566386, 820652717, 626379552, 2694287, 1068893872, 402591187, 393569177, 315129706, 515096595, 472665278, 107466266, 885195181, 600854427, 808826892, 1069447946, 582747524, 75190884, 855091317, 26584886, 734114269, 680342077, 381922143, 947219285, 818112506]
a0: 10306258447
Density: 1.0001927332430454
Solution Weight: 15
Solution: [0 1 1 1 1 0 0 1 1 0 0 0 0 1 1 1 0 0 0 1 1 1 0 1 0 1 0 0 0 1]
N: 7
################################# STEP 2 - Construct lattice #################################
Lattice:
[[          2           0           0 ...  4584768195           0
            7]
 [          0           2           0 ...  6305072592           0
            7]
 [          0           0           2 ...  3110734942           0
            7]
 ...
 [          0           0           0 ...  6630534995   